# Exploratory Data Analysis

Sau khi hoàn thành phần tiền xử lí dữ liệu, dữ liệu đã được làm sạch giúp ta trả lời một số câu hỏi chính xác và rõ ràng hơn.  
##### Đầu tiên, import các thư viện cần thiết

In [2]:
import pandas as pd

##### Load data

In [3]:
steam_df = pd.read_csv('processed_data.csv')
steam_df.head()

,Title,withDLC,isMature,Developer,Publisher,PositiveReviews,TotalReviews,NegativeReviews,Price,ReleaseYear,...,Genre__RPG,Genre__Racing,Genre__Simulation,Genre__Software Training,Genre__Sports,Genre__Strategy,Genre__Utilities,Genre__Video Production,Genre__Web Publishing,Rating
0,Endless Monday: Dreams and Deadlines,0,0,hcnone,hcnone,737,737,0,142000.0,2023,...,0,0,0,0,0,0,0,0,0,0.931514
1,Papa's Freezeria Deluxe,0,0,Flipline Studios,Flipline Studios,6514,6562,48,87500.0,2023,...,0,0,1,0,0,1,0,0,0,0.957730
2,Cats Hidden in Jingle Jam,0,0,Travellin Cats,Travellin Cats,5097,5108,11,0.0,2023,...,0,0,0,0,0,0,0,0,0,0.959760
3,An Arcade Full of Cats,1,0,Devcats,Devcats,3018,3045,27,0.0,2023,...,0,0,0,0,0,0,0,0,0,0.947230
4,Ib,0,0,kouri,PLAYISM,3068,3098,30,150000.0,2022,...,0,0,0,0,0,0,0,0,0,0.946714


#### Câu hỏi 1: Nhà phát triển game nào có tiềm năng phát triển nhất?
Để trả lời câu hỏi này trước tiên ta sẽ xem xét có bao nhiêu nhà phát triển trong bộ dữ liệu.   
Sau đó chọn ra các nhà phát triển có các game ở mức rating cao và xem xét các game từ trước đến hiện tại của các developer đó và vẽ biểu đồ thể hiện xu hướng.

Xem xét các developer:

In [63]:
dev_df = steam_df['Developer'].value_counts()

# Number of dev
len(dev_df)

2118

Có khá nhiều Developer, để dễ dàng quan sát và phân tích ta chỉ lấy top 10 nhà phát triển đã phát triển được nhiều game nhất để phân tích.  
Sau đó ta xét các năm ra mắt game của top 10 nhà phát triển để được một mảng `list_year`


In [80]:
# Top 10 Developer
top10_dev_df = dev_df.nlargest(10)
top10_dev_df

# Oldest and newest release year
releaseY = steam_df[steam_df['Developer'].isin(top10_dev_df.index)]
# minY = releaseY.min()
# maxY = releaseY.max()

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(releaseY[['Developer', 'ReleaseYear']])

                    Developer  ReleaseYear
15                   上海アリス幻樂団         2007
27                   上海アリス幻樂団         2017
47                   上海アリス幻樂団         2021
56                   上海アリス幻樂団         2015
76         PopCap Games, Inc.         2007
83        Jackbox Games, Inc.         2016
95        Jackbox Games, Inc.         2020
107              Nihon Falcom         2015
117        PopCap Games, Inc.         2006
131                  上海アリス幻樂団         2019
133   Humongous Entertainment         1996
160        PopCap Games, Inc.         2006
164        PopCap Games, Inc.         2010
202        PopCap Games, Inc.         2008
308       Jackbox Games, Inc.         2019
319        PopCap Games, Inc.         2009
321               Square Enix         2023
340       Jackbox Games, Inc.         2015
393                  上海アリス幻樂団         2013
429         Kairosoft Co.,Ltd         2023
431                  上海アリス幻樂団         2010
436   Humongous Entertainment         1995
441        